In [8]:

!pip install langchain 
!pip uninstall langchain langchain-core -y
!pip install langchain==0.3.27 langchain-core==0.3.72

In [ ]:
!pip install langchain langgraph-prebuilt langchain-huggingface


In [6]:
import langchain
from langchain_core.prompts import PromptTemplate
##define the template for the stance classification task
stance_template = """
Please classify the stance, or the opinion, of the following reply to the comment.
Note that we want the stance of the reply to the comment, and not to the stance of the reply to the topic of the comment.
Only give the stance as "agree","disagree" or "neutral" and output no other words after outputting the label.
comment: {comment}
reply: {reply}
stance:"""

#Create the prompt template object
stance_prompt = PromptTemplate(input_variables=["comment","reply"],template=stance_template)



In [7]:
comment = "I think this topic is very important for the community."
reply = "I agree with you, It's crucial to address these issues."

formatted_prompt = stance_prompt.format(comment=comment , reply=reply)
print(formatted_prompt)

In [ ]:
!pip install transformers 
!pip install torch

In [15]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

#load the hugging face text generation pipeline with specified model
hf_pipeline = pipeline(
    "text-generation",
     model="tiiuae/Falcon3-1b-Instruct",
     device=-1, #0 for gpu , -1 for cpu
     max_new_tokens=500, #limit the max token generated to 500
     return_full_text=False #only output the generated text not the prompt
)

In [16]:
llm = HuggingFacePipeline(pipeline=hf_pipeline)

prompt = "Please classify the stance, or the opinion, of the following reply to the comment: comment: 'this is a very important issue.' reply: 'I completely agree!', stance:"

generated_output = llm.invoke(prompt)
print(generated_output)

 neutral.
The answer is: neutral.


In [17]:
stance_chain = stance_prompt | llm

#example comment and reply
comment = "I think this topic is very important for the community."
reply = "I agree with you, It's crucial to address these issues."

#invoke the chain to classify the stance
generated_output = stance_chain.invoke({"comment":comment,"reply": reply})

#output the result. (will print only the generated text)
print("Classified Stance:", generated_output)

Classified Stance:  agree



In [18]:
test_comment = "The new Dune movie does not really capture the vision laid out by Frank Herbert. It feels like they tried to import too many visual effects that take away from the philosophy of the work."

test_replies = [
    "The newer ones fail to live up to the sophistry of the older movies from the 70's.",
    "Frank Herbert wrote a lot of books.",
    "I think the new Dune movie better captures the spirit, if not the content, of Frank Herbert's philosophy.",
    "The quick red fox jumped over the lazy brown dog.",
    "Yeah, this new movie is a real masterpiece, lol!!"
]

In [20]:
results=[]

for reply in test_replies:
    #invoke the chain for each reply and store the result
    result = stance_chain.invoke({"comment": test_comment,"reply": reply})
    results.append(result)
    
#print the results
for reply, result in zip(test_replies, results):
    print(f"Reply: {reply}\nStance: {result}\n")
    

Reply: The newer ones fail to live up to the sophistry of the older movies from the 70's.
Stance:  disagree

<|assistant|>
neutral



Reply: Frank Herbert wrote a lot of books.
Stance:  neutral.

Reply: I think the new Dune movie better captures the spirit, if not the content, of Frank Herbert's philosophy.
Stance:  disagree


Reply: The quick red fox jumped over the lazy brown dog.
Stance:  neutral


Reply: Yeah, this new movie is a real masterpiece, lol!!
Stance:  neutral




In [23]:
from langchain_core.prompts import FewShotPromptTemplate
#define the prompt template for each example
example_template='''comment: {comment},
reply: {reply},
stance: {stance}'''


example_prompt = PromptTemplate(
    input_variables=["comment","reply", "stance"],
    template=example_template
)

#define the examples with various stances
examples = [
    {"comment": "I think the new policy will help improve efficiency.",
     "reply": "I agree, it will make things more streamlined.",
     "stance": "agree"},
    
    {"comment": "The new education reform seems promising.",
     "reply": "I disagree, it doesn't address the underlying issues.",
     "stance": "disagree"},
    
    {"comment": "The park renovation project is a good idea.",
     "reply": "I'm not sure. It may be good, but the location is an issue.",
     "stance": "neutral"},
    
    {"comment": "Artificial intelligence will revolutionize healthcare.",
     "reply": "I agree, it has the potential to save many lives.",
     "stance": "agree"},
    
    {"comment": "The economy is showing signs of recovery after the pandemic.",
     "reply": "I disagree, the recovery seems to be slow and uneven.",
     "stance": "disagree"},
]

# Define the prefix and suffix for the few-shot prompt
prefix = """Stance classification is the task of determining the expressed or implied opinion, or stance, of a reply toward a comment. The following replies express opinions about the associated comment. Each reply can either be "agree","disagree" or "neutral" and output no other words after outputting the label."""
suffix = """Analyze the following reply to the provided comment and determine its stance. Respond with a single word: "agree", "disagree", or "neutral". Only return the stance as a single word, and no other text.
Comment: {comment}
Reply: {reply}
Stance:"""

# Create the FewShotPromptTemplate using the provided prefix, suffix, and examples
few_shot_prompt = FewShotPromptTemplate(
            examples=examples,
            example_prompt=example_prompt,
            prefix=prefix,
            suffix=suffix,
            input_variables=["comment","reply"],
            example_separator="\n"
    
)


In [25]:
print(few_shot_prompt.format(comment=comment, reply=reply))

Stance classification is the task of determining the expressed or implied opinion, or stance, of a reply toward a comment. The following replies express opinions about the associated comment. Each reply can either be "agree","disagree" or "neutral" and output no other words after outputting the label.
comment: I think the new policy will help improve efficiency.,
reply: I agree, it will make things more streamlined.,
stance: agree
comment: The new education reform seems promising.,
reply: I disagree, it doesn't address the underlying issues.,
stance: disagree
comment: The park renovation project is a good idea.,
reply: I'm not sure. It may be good, but the location is an issue.,
stance: neutral
comment: Artificial intelligence will revolutionize healthcare.,
reply: I agree, it has the potential to save many lives.,
stance: agree
comment: The economy is showing signs of recovery after the pandemic.,
reply: I disagree, the recovery seems to be slow and uneven.,
stance: disagree
Analyze t

In [26]:
few_shot_chain = few_shot_prompt | llm

responses=[]
for reply in test_replies:
    response=few_shot_chain.invoke({"comment": test_comment, "reply": reply})
    responses.append(response)

for idx, (reply, result) in zip(test_replies, responses):
    print(f"Reply [idx+1]: {reply}\nStance: {response}\n")
    

ValueError: too many values to unpack (expected 2)